## Read in a whole recording

In [31]:
import os
import scipy.io
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import load_intan_rhd_format as intan
from AG_functions import *

In [32]:
folder_path = '../../../../../media/ela2X/intanData/Stargazer/0056/rawData';
file_type = '.rhd';

## Concatenate downsampled data from every file to get the total data from one recording session

In [3]:
# downsampling factor
alpha = 234;

allData = loadRecording(folder_path, file_type, alpha)



File 1 : ../../../../../media/ela2X/intanData/Stargazer/0056/rawData/stg110_20240125_240125_125219.rhd

Reading Intan Technologies RHD Data File, Version 3.0

Found 256 amplifier channels.
Found 12 auxiliary input channels.
Found 0 supply voltage channels.
Found 2 board ADC channels.
Found 2 board digital input channels.
Found 0 board digital output channels.
Found 0 temperature sensors channels.

File contains 300.002 seconds of data.  Amplifiers were sampled at 30.00 kS/s.

Allocating memory for data...
Reading data from file...
10% done...
20% done...
30% done...
40% done...


KeyboardInterrupt: 

In [ ]:
allData.to_csv('ela2X_intanData_Stargazer_0056_rawData_128Hz.csv', index_label = 'Time (s)')

In [33]:
allData = pd.read_csv('ela2X_intanData_Stargazer_0056_rawData_128Hz.csv', index_col=0)
allData

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
Time(s),,,,,,,,,,,,,,,,,,,,,
0.0000,-264.225,-240.435,-234.000,-190.710,-225.420,-201.630,-195.000,-197.730,-178.815,-144.690,...,-130.260,-130.845,-173.355,-182.715,-157.755,-165.555,-172.185,-163.800,-185.445,-217.815
0.0078,-411.255,-288.990,-271.635,-229.515,-246.090,-223.860,-247.260,-266.565,-222.495,-221.520,...,-212.745,-205.920,-208.650,-216.645,-193.440,-188.370,-206.895,-197.340,-232.245,-315.120
0.0156,-213.915,-171.795,-140.985,-130.650,-171.210,-177.450,-153.075,-165.360,-137.475,-131.235,...,-146.835,-139.035,-166.920,-166.140,-137.085,-125.580,-139.035,-159.705,-185.640,-210.990
0.0234,-103.155,-128.895,-129.090,-95.745,-123.825,-136.695,-127.335,-144.885,-126.165,-103.350,...,-129.285,-137.475,-121.290,-113.295,-95.160,-84.435,-74.880,-103.350,-110.760,-111.735
0.0312,-127.920,-130.260,-158.145,-134.940,-101.400,-93.990,-106.275,-106.860,-118.170,-97.110,...,-118.950,-123.630,-100.425,-97.500,-77.610,-52.065,-60.840,-72.345,-70.980,-56.355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3601.8996,-172.770,-129.090,-135.720,-106.080,-80.535,-72.540,-91.455,-73.320,-85.800,-129.090,...,-101.010,-83.070,-38.610,-43.875,-19.305,1.560,-44.070,-16.575,-47.580,-140.010
3601.9074,-21.255,-43.095,-50.115,-32.565,-6.825,-5.460,-25.155,-3.705,-16.575,-55.575,...,-36.465,-36.270,-12.675,-8.775,1.365,25.935,-2.535,-6.825,-16.185,-59.085
3601.9152,-47.775,-89.895,-119.925,-113.295,-57.135,-50.310,-93.990,-63.180,-56.355,-65.130,...,-75.660,-80.535,-61.815,-59.085,-46.800,-1.755,-46.995,-47.385,-52.065,-74.295


# Label Records as non-ictal, pre-ictal and ictal

### Load Seizure Start and End Times

In [34]:
starts = scipy.io.loadmat("seizureStartTimes.mat")['seizureStartTimes']
print(np.shape(starts))

ends = scipy.io.loadmat("seizureEndTimes.mat")
ends = ends['seizureEndTimes'].reshape(-1,1)
print(np.shape(ends))

(73, 1)
(73, 1)


### What's the min, max, average, std dev of the time between seizures?
Use to choose what to classify pre-ictal time as.

In [35]:
inter_ictal_periods = np.diff(starts.reshape(1,-1))
print("Inter-ictal period Stats\nMean:", round(inter_ictal_periods.mean()))
print("Min:", np.floor(inter_ictal_periods.min()))
print("Max:", round(inter_ictal_periods.max()))
print("Std dev:", round(inter_ictal_periods.std()))

Inter-ictal period Stats
Mean: 49
Min: 5.0
Max: 595
Std dev: 85


# Choose the class definitions

0. Non-seizure
1. Pre-seizure
2. Seizure

In [36]:
label_data(allData, starts, ends, np.floor(inter_ictal_periods.min()))
allData

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,class
Time(s),,,,,,,,,,,,,,,,,,,,,
0.0000,-264.225,-240.435,-234.000,-190.710,-225.420,-201.630,-195.000,-197.730,-178.815,-144.690,...,-130.845,-173.355,-182.715,-157.755,-165.555,-172.185,-163.800,-185.445,-217.815,0
0.0078,-411.255,-288.990,-271.635,-229.515,-246.090,-223.860,-247.260,-266.565,-222.495,-221.520,...,-205.920,-208.650,-216.645,-193.440,-188.370,-206.895,-197.340,-232.245,-315.120,0
0.0156,-213.915,-171.795,-140.985,-130.650,-171.210,-177.450,-153.075,-165.360,-137.475,-131.235,...,-139.035,-166.920,-166.140,-137.085,-125.580,-139.035,-159.705,-185.640,-210.990,0
0.0234,-103.155,-128.895,-129.090,-95.745,-123.825,-136.695,-127.335,-144.885,-126.165,-103.350,...,-137.475,-121.290,-113.295,-95.160,-84.435,-74.880,-103.350,-110.760,-111.735,0
0.0312,-127.920,-130.260,-158.145,-134.940,-101.400,-93.990,-106.275,-106.860,-118.170,-97.110,...,-123.630,-100.425,-97.500,-77.610,-52.065,-60.840,-72.345,-70.980,-56.355,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3601.8996,-172.770,-129.090,-135.720,-106.080,-80.535,-72.540,-91.455,-73.320,-85.800,-129.090,...,-83.070,-38.610,-43.875,-19.305,1.560,-44.070,-16.575,-47.580,-140.010,0
3601.9074,-21.255,-43.095,-50.115,-32.565,-6.825,-5.460,-25.155,-3.705,-16.575,-55.575,...,-36.270,-12.675,-8.775,1.365,25.935,-2.535,-6.825,-16.185,-59.085,0
3601.9152,-47.775,-89.895,-119.925,-113.295,-57.135,-50.310,-93.990,-63.180,-56.355,-65.130,...,-80.535,-61.815,-59.085,-46.800,-1.755,-46.995,-47.385,-52.065,-74.295,0


In [37]:
# check it worked by viewing the average voltage of each class
allData.set_index('class').stack().groupby(level='class').mean().to_frame()

,0
class,
0,-0.015083
1,-0.737252
2,0.231983


Hmm, that's encouraging that the classes look different already just based on average raw voltage.

## Consider denoising the data various ways. not sure how

## Do sliding window and labels

In [38]:
allData.iloc[0:100, -1].mode()

0    0
Name: class, dtype: int64

In [89]:
# how long in seconds you want each window to be
window_size = 5
# the sample/sec sampling freqency of the (downsampled) dataset
data_hz = 128
samples_per_window = data_hz * 5
# how many records to move ahead when making the next window
step_size = 100
    
iters = (len(allData) - samples_per_window) // step_size + 1
    
allData_labels_1d = np.zeros(len(allData))
window_labels =[]
counter = 0
start_counter = 0
for start_window in range(0, len(allData) - samples_per_window + 1, step_size):
    end_window = start_window + window_size
    end_counter = start_counter + samples_per_window

    label = allData.iloc[start_window:end_window, -1].mode()[0]
    # create "answer" y vector
    window_labels.append(label)

    # create 1D vector with same length as allData to create a 3rd dimension for allData (after this loop finishes)
    allData_labels_1d[start_counter:end_counter] = counter
    counter = counter + 1
    start_counter = end_counter

In [ ]:
round(allData.shape[0]/temp.shape[0])

In [86]:
samples_per_window

640

In [82]:
start_counter

7040

In [83]:
end_counter

7040

In [84]:
counter

11

In [87]:
pd.DataFrame(allData_labels_1d)[0:2006]

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0


In [97]:
round(allData.shape[0]/temp.shape[0])

640

In [92]:
#pd.set_option('display.max_rows', None)
temp = pd.DataFrame(pd.Series(allData_labels_1d).value_counts()).sort_index()
temp

,count
0.0,640
1.0,640
2.0,640
3.0,640
4.0,640
5.0,640
6.0,640
7.0,640
8.0,640
9.0,640


In [17]:
# add 3rd dimension to allData that keeps track of which labeled block each sample is in
# make the 1D vector 2D (each element in the 1D vector becomes it's own row)
allData_labels_2d = np.expand_dims(allData_labels_1d, axis=1)

# make the number of columns in new 2D vector match the number of columns in allData by copying the single element in each
# row currently, as many times as needed.
allData_labels_expanded = np.repeat(allData_labels_2d, allData.shape[1], axis=1)

# now can add the window number to allData as a 3rd dimension
allData_3d = np.dstack((allData, allData_labels_expanded))

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,class
Time(s),,,,,,,,,,,,,,,,,,,,,
0.0000,-264.225,-240.435,-234.000,-190.710,-225.420,-201.630,-195.000,-197.730,-178.815,-144.690,...,-130.845,-173.355,-182.715,-157.755,-165.555,-172.185,-163.800,-185.445,-217.815,0
0.0078,-411.255,-288.990,-271.635,-229.515,-246.090,-223.860,-247.260,-266.565,-222.495,-221.520,...,-205.920,-208.650,-216.645,-193.440,-188.370,-206.895,-197.340,-232.245,-315.120,0
0.0156,-213.915,-171.795,-140.985,-130.650,-171.210,-177.450,-153.075,-165.360,-137.475,-131.235,...,-139.035,-166.920,-166.140,-137.085,-125.580,-139.035,-159.705,-185.640,-210.990,0
0.0234,-103.155,-128.895,-129.090,-95.745,-123.825,-136.695,-127.335,-144.885,-126.165,-103.350,...,-137.475,-121.290,-113.295,-95.160,-84.435,-74.880,-103.350,-110.760,-111.735,0
0.0312,-127.920,-130.260,-158.145,-134.940,-101.400,-93.990,-106.275,-106.860,-118.170,-97.110,...,-123.630,-100.425,-97.500,-77.610,-52.065,-60.840,-72.345,-70.980,-56.355,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3601.8996,-172.770,-129.090,-135.720,-106.080,-80.535,-72.540,-91.455,-73.320,-85.800,-129.090,...,-83.070,-38.610,-43.875,-19.305,1.560,-44.070,-16.575,-47.580,-140.010,0
3601.9074,-21.255,-43.095,-50.115,-32.565,-6.825,-5.460,-25.155,-3.705,-16.575,-55.575,...,-36.270,-12.675,-8.775,1.365,25.935,-2.535,-6.825,-16.185,-59.085,0
3601.9152,-47.775,-89.895,-119.925,-113.295,-57.135,-50.310,-93.990,-63.180,-56.355,-65.130,...,-80.535,-61.815,-59.085,-46.800,-1.755,-46.995,-47.385,-52.065,-74.295,0


In [ ]:
pd.Series(window_labels).value_counts()

In [ ]:
y.value_counts()/pd.Series(window_labels).value_counts()

In [ ]:
np.shape(allData_labels)

In [ ]:
np.shape(allData)

In [ ]:
labels_2d = np.expand_dims(allData_labels, axis=1)
labels_2d

In [ ]:
labels_3d = np.repeat(labels_2d, allData.shape[1], axis=1)
labels_3d

In [ ]:
allData_3d = np.dstack((allData, labels_3d))
allData_3d

[i like this paper](https://pmc.ncbi.nlm.nih.gov/articles/PMC11021591/#sec2)

In [ ]:
# Description:
# Sections dataset into windows of user defined size and separation from each other and then creates labels for each section.
# The chosen label is defined as the class mode of the samples in the window. 
# Then a 3rd dimension is added to allData that keeps track of which window each records is a part of
# Returns: numpy array of window labels and a 3d array that's allData + window block marker/groupings/counter
# Parameters:
# allData: the whole (downsampled) dataset you want to make window labels for.
    # The last column in this dataset MUST be "class" with the class labels for each record
# window_size: how long in seconds you want each window to be
# step_size: how many records to move ahead when making the next window
# data_hz: the sample/sec sampling freqency of the (downsampled) dataset
def windows(allData, window_size, data_hz, step_size):
    samples_per_window = data_hz * 5
    
    iters = (len(allData) - samples_per_window) // step_size + 1
    
    allData_labels_1d = np.zeros(len(allData))
    window_labels =[]
    counter = 0
    for start in range(0, len(allData) - samples_per_window + 1, step_size):
        end = start + window_size

        label = allData.iloc[start:end, -1].mode()[0]
        # create "answer" y vector
        window_labels.append(label)

        # create 1D vector with same length as allData to create a 3rd dimension for allData (after this loop finishes)
        allData_labels_1d[start:end] = counter
        counter = counter + 1

    # add 3rd dimension to allData that keeps track of which labeled block each sample is in
    # make the 1D vector 2D (each element in the 1D vector becomes it's own row)
    allData_labels_2d = np.expand_dims(allData_labels_1d, axis=1)

    # make the number of columns in new 2D vector match the number of columns in allData by copying the single element in each
    # row currently, as many times as needed.
    allData_labels_expanded = np.repeat(allData_labels_2d, allData.shape[1], axis=1)

    # now can add the window number to allData as a 3rd dimension
    allData_3d = np.dstack((allData, allData_labels_expanded))

    return window_labels, allData_3d

In [ ]:
allData_labels_expanded

In [ ]:
window_size = 5
data_hz = 128
step_size = 100

window_labels, allData_3d = windows(allData, window_size, data_hz, step_size)

In [ ]:
# Check unique window labels in the third dimension
unique_windows = np.unique(allData_3d[:, :, -1])
print("Unique window labels in third dimension:", unique_windows.astype(int))

In [ ]:
samples_per_window = data_hz * 5
iters = (len(allData) - samples_per_window) // step_size + 1
iters

In [ ]:
len(allData_3d[:, :, -1])/iters

In [ ]:
pd.Series(window_labels).value_counts()

## Train, val, test split data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
window_labels = pd.Series(window_labels).reset_index(name='class').rename(columns={'index': 'block'})

In [ ]:
window_labels

In [ ]:
# 70-20-10 train-val-test
# can't pass allData_3d into this function because it would split up the window blocks
# can only pass in the list of only unique window blocks
y_train, y_val_temp = train_test_split(window_labels, test_size=0.3, random_state=42)
y_val, y_test = train_test_split(y_val_temp, test_size=1/3, random_state=42)

In [ ]:
tl = len(y_train)
vl = len(y_val)
testl = len(y_test)
suml = tl+vl+testl
print("train:", tl, round((tl/suml)*100),"%", "\nval:", vl,round((vl/suml)*100),"%", "\ntest:", testl, round((testl/suml)*100),"%",)

In [ ]:
y_train['block']

In [ ]:
pd.Series(unique_windows).value_counts()

In [ ]:
# Step 1: Check unique window labels in the third dimension
unique_windows = np.unique(allData_3d[:, :, -1])
print("Unique window labels in third dimension:", unique_windows.astype(int))

In [ ]:
train_mask = np.isin(allData_3d[:, 0, -1], y_train['block'])
val_mask = np.isin(allData_3d[:, 0, -1], y_val['block'])
test_mask = np.isin(allData_3d[:, 0, -1], y_test['block'])

In [ ]:
x_train = allData_3d[train_mask]
x_val = allData_3d[val_mask]
x_test = allData_3d[test_mask]

In [ ]:
x_train.astype(int)

In [ ]:
np.unique(x_train[:, 0, -1])